In [1]:
import os
import pandas as pd
import numpy as np
from dash import Dash, html, dcc, Input, Output, no_update
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
try:
    from scipy import stats
    HAVE_KDE = True
except Exception:
    HAVE_KDE = False

In [2]:
# -------------------------------
# Data Loader
# -------------------------------
def load_csv_any(paths):
    for p in paths:
        if os.path.exists(p):
            return pd.read_csv(p)
    raise FileNotFoundError(paths)

matches = load_csv_any(["../data/cleaned/matches.csv", "./matches.csv"])
deliveries = load_csv_any(["../data/cleaned/deliveries.csv", "./deliveries.csv"])

matches["Season"] = pd.to_numeric(matches["Season"], errors="coerce").astype("Int64")

In [3]:

# -------------------------------
# Theme & Team Colors
# -------------------------------
COLORS = {
    "bg_dark": "#0b1020",
    "card_bg": "#111827",
    "accent": "#ffb703",
    "text": "white"
}

TEAM_INFO = {
    "Chennai Super Kings": {"abbr": "CSK", "color": "#F9CD05"},
    "Mumbai Indians": {"abbr": "MI", "color": "#004BA0"},
    "Royal Challengers Bangalore": {"abbr": "RCB", "color": "#DA1818"},
    "Kolkata Knight Riders": {"abbr": "KKR", "color": "#3A225D"},
    "Delhi Capitals": {"abbr": "DC", "color": "#17449B"},
    "Delhi Daredevils": {"abbr": "DD", "color": "#17449B"},
    "Sunrisers Hyderabad": {"abbr": "SRH", "color": "#FF822A"},
    "Rajasthan Royals": {"abbr": "RR", "color": "#E73895"},
    "Punjab Kings": {"abbr": "PBKS", "color": "#C91C1C"},
    "Kings XI Punjab": {"abbr": "KXIP", "color": "#C91C1C"},
    "Lucknow Super Giants": {"abbr": "LSG", "color": "#009FDB"},
    "Gujarat Titans": {"abbr": "GT", "color": "#1C2833"},
    "Rising Pune Supergiant": {"abbr": "RPS", "color": "#652D91"},
    "Pune Warriors": {"abbr": "PWI", "color": "#3C8DBC"},
    "Kochi Tuskers Kerala": {"abbr": "KTK", "color": "#EE7B30"},
    "Deccan Chargers": {"abbr": "DCG", "color": "#2E8B57"}
}

team_cols = ["Team1", "Team2", "Winner", "Toss_Winner"]
teams = set()
for col in team_cols:
    teams.update(matches[col].dropna().unique())
invalid = {"No Result", "No Toss", " ", "", "nan"}
team_list = sorted([t for t in teams if t not in invalid])

season_min, season_max = int(matches["Season"].min()), int(matches["Season"].max())

# -------------------------------
# Dash App
# -------------------------------
app = Dash(__name__, suppress_callback_exceptions=True)
server = app.server

app.layout = html.Div(
    style={"backgroundColor": COLORS["bg_dark"], "color": COLORS["text"], "padding": "20px"},
    children=[
        html.H2("Head to Head",
                style={"textAlign": "center", "color": COLORS["accent"]}),

        html.Div([
            html.Div([
                html.Label("Team A"),
                dcc.Dropdown(id='team-a', options=[{"label": t, "value": t} for t in team_list],
                             placeholder="Choose Team A", style={"color": "black"})
            ], style={"width": "45%", "display": "inline-block", "marginRight": "5%"}),

            html.Div([
                html.Label("Team B"),
                dcc.Dropdown(id='team-b', options=[{"label": t, "value": t} for t in team_list],
                             placeholder="Choose Team B", style={"color": "black"})
            ], style={"width": "45%", "display": "inline-block"}),
        ]),

        html.Br(),
        html.Label("Select Season Range"),
        dcc.RangeSlider(id='season-range', min=season_min, max=season_max, step=1,
                        value=[season_min, season_max],
                        marks={y: str(y) for y in range(season_min, season_max + 1)},
                        tooltip={"placement": "bottom"}),

        html.Div(id="kpi-row", style={"display": "flex", "flexWrap": "wrap", "marginTop": "10px"}),
        html.Div(id="plots-container", style={"marginTop": "20px", "display": "flex",
                                              "flexDirection": "column", "gap": "25px"})
    ]
)

# -------------------------------
# Helpers
# -------------------------------
def team_abbr(name):
    return TEAM_INFO.get(name, {}).get("abbr", str(name)[:3].upper())

def team_color(name, fallback="#999"):
    return TEAM_INFO.get(name, {}).get("color", fallback)

# -------------------------------
# Callback
# -------------------------------
@app.callback(
    [Output("kpi-row", "children"),
     Output("plots-container", "children")],
    [Input("team-a", "value"), Input("team-b", "value"), Input("season-range", "value")]
)
def update_tab2(team_a, team_b, season_range):
    if not season_range:
        return [no_update] * 2

    s0, s1 = season_range
    df = matches.copy()

    # Filter teams
    if team_a and team_b:
        df = df[((df["Team1"] == team_a) & (df["Team2"] == team_b)) |
                ((df["Team1"] == team_b) & (df["Team2"] == team_a))]
    elif team_a:
        df = df[(df["Team1"] == team_a) | (df["Team2"] == team_a)]
    elif team_b:
        df = df[(df["Team1"] == team_b) | (df["Team2"] == team_b)]

    df = df[(df["Season"] >= s0) & (df["Season"] <= s1)]
    if df.empty:
        return html.Div("No matches found for this range.", style={"textAlign": "center"}), None

    deliv = deliveries[deliveries["Match_Id"].isin(df["Id"])].copy()

    # -------------------------------
    # KPI Row
    # -------------------------------
    total = len(df)
    ties = (df["Result"] == "Tie").sum()
    nr = (df["Result"] == "No Result").sum()
    t1_wins = (df["Winner"] == team_a).sum() if team_a else 0
    t2_wins = (df["Winner"] == team_b).sum() if team_b else 0

    def kpi(label, val, color):
        return html.Div([
            html.Div(label, style={"fontSize": "12px", "color": COLORS["accent"]}),
            html.H4(str(val), style={"margin": "0", "color": color})
        ], style={"background": COLORS["card_bg"], "border": f"1px solid {color}",
                  "borderRadius": "8px", "padding": "8px", "margin": "4px",
                  "textAlign": "center", "minWidth": "120px"})

    kpis = html.Div([
        kpi("Matches", total, COLORS["text"]),
        kpi(f"{team_abbr(team_a)} Wins" if team_a else "Team A Wins", t1_wins, team_color(team_a)),
        kpi(f"{team_abbr(team_b)} Wins" if team_b else "Team B Wins", t2_wins, team_color(team_b)),
        kpi("Ties", ties, "#fdd835"),
        kpi("No Result", nr, "#94a3b8"),
    ], style={"display": "flex", "justifyContent": "center", "flexWrap": "wrap"})

    plots = []

    # -------------------------------
    # Win Comparison + Toss Gauge
    # -------------------------------
    color_a, color_b = team_color(team_a), team_color(team_b)
    win_fig = px.bar(x=[team_abbr(team_a), team_abbr(team_b)],
                     y=[t1_wins, t2_wins],
                     color=[team_a, team_b],
                     color_discrete_sequence=[color_a, color_b],
                     title="Win Comparison")
    win_fig.update_layout(height=260, plot_bgcolor=COLORS["bg_dark"],
                          paper_bgcolor=COLORS["bg_dark"], font_color="white")

    toss_win = (df["Toss_Winner"] == df["Winner"]).mean() * 100
    toss_fig = go.Figure(go.Indicator(
        mode="gauge+number", value=toss_win,
        gauge={"axis": {"range": [0, 100]}, "bar": {"color": COLORS["accent"]}},
        title={"text": "Toss → Match Win %"}
    ))
    toss_fig.update_layout(height=260, plot_bgcolor=COLORS["bg_dark"],
                           paper_bgcolor=COLORS["bg_dark"], font_color="white")

    plots.append(html.Div([
        html.Div(dcc.Graph(figure=win_fig), style={"width": "48%", "display": "inline-block"}),
        html.Div(dcc.Graph(figure=toss_fig), style={"width": "48%", "display": "inline-block", "marginLeft": "2%"})
    ]))

    # -------------------------------
    # Season-wise Wins Trend
    # -------------------------------
    st = df.groupby(["Season", "Winner"]).size().unstack(fill_value=0)
    fig_trend = go.Figure()
    for col in st.columns:
        clr = team_color(col)
        fig_trend.add_trace(go.Scatter(x=st.index, y=st[col], mode="lines+markers",
                                       name=team_abbr(col), line=dict(color=clr)))
    fig_trend.update_layout(title="Season-wise Wins", height=280, plot_bgcolor=COLORS["bg_dark"],
                            paper_bgcolor=COLORS["bg_dark"], font_color="white",
                            xaxis=dict(dtick=1, showgrid=False),
                            yaxis=dict(gridcolor="rgba(255,255,255,0.05)"))
    plots.append(dcc.Graph(figure=fig_trend))

    # -------------------------------
    # Match Margin Distribution
    # -------------------------------
    df["Result_Margin"] = pd.to_numeric(df["Result_Margin"], errors="coerce").fillna(0)
    df["Margin_Type"] = df["Result"].apply(lambda r: "Runs" if r == "Runs" else "Wickets" if r == "Wickets" else None)
    df = df[df["Margin_Type"].notna()]
    runs_df = df[df["Margin_Type"] == "Runs"]
    wkts_df = df[df["Margin_Type"] == "Wickets"]

    fig_margin = make_subplots(rows=1, cols=2, shared_yaxes=True,
                               subplot_titles=("Defending Wins (By Runs)", "Chasing Wins (By Wickets)"))
    fig_margin.add_trace(go.Histogram(x=runs_df["Result_Margin"], nbinsx=15, marker_color="#ffb703"), 1, 1)
    if HAVE_KDE and len(runs_df) > 5:
        kde_x = np.linspace(0, runs_df["Result_Margin"].max(), 100)
        kde_y = stats.gaussian_kde(runs_df["Result_Margin"]).pdf(kde_x)
        kde_y_scaled = kde_y * len(runs_df) * (kde_x[1] - kde_x[0]) * 5
        fig_margin.add_trace(go.Scatter(x=kde_x, y=kde_y_scaled, mode='lines', line=dict(color="#ffb703")), 1, 1)

    fig_margin.add_trace(go.Histogram(x=wkts_df["Result_Margin"], nbinsx=10, marker_color="#00b4d8"), 1, 2)
    if HAVE_KDE and len(wkts_df) > 5:
        kde_x2 = np.linspace(0, wkts_df["Result_Margin"].max(), 100)
        kde_y2 = stats.gaussian_kde(wkts_df["Result_Margin"]).pdf(kde_x2)
        kde_y2_scaled = kde_y2 * len(wkts_df) * (kde_x2[1] - kde_x2[0]) * 5
        fig_margin.add_trace(go.Scatter(x=kde_x2, y=kde_y2_scaled, mode='lines', line=dict(color="#00b4d8")), 1, 2)

    fig_margin.update_layout(title="Match Margin Distribution — Defending vs Chasing",
                             height=350, plot_bgcolor=COLORS["bg_dark"], paper_bgcolor=COLORS["bg_dark"],
                             font_color="white", showlegend=False)
    plots.append(dcc.Graph(figure=fig_margin))

# -------------------------------
# 🏅 Top 3 Batters & Bowlers (Team Colors)
# -------------------------------
# 🏏 Top 3 Batters
    top_bats = (
        deliv.groupby(["Batter", "Batting_Team"])["Batsman_Runs"]
        .sum()
        .reset_index()
        .nlargest(3, "Batsman_Runs")
        .sort_values("Batsman_Runs", ascending=True)  # ensures top scorer is at top in horizontal bar
    )
    top_bats["display"] = top_bats.apply(lambda r: f"{r['Batter']} — {team_abbr(r['Batting_Team'])}", axis=1)
    top_bats["color"] = top_bats["Batting_Team"].map(lambda t: team_color(t))

    fig_bat = go.Figure(
        go.Bar(
            x=top_bats["Batsman_Runs"],
            y=top_bats["display"],
            orientation="h",
            marker_color=top_bats["color"],
            text=top_bats["Batsman_Runs"],
            textposition="outside"
        )
    )
    fig_bat.update_layout(
        title="Top 3 Batters by Runs",
        height=260,
        plot_bgcolor=COLORS["bg_dark"],
        paper_bgcolor=COLORS["bg_dark"],
        font_color="white",
        xaxis_title="Runs",
        yaxis_title=None,
        margin=dict(l=60, r=20, t=40, b=20),
        title_x = 0.5
    )

    # 🎯 Top 3 Bowlers
    top_bowl = (
        deliv[deliv.get("Is_Wicket", 0) == 1]
        .groupby(["Bowler", "Bowling_Team"])
        .size()
        .reset_index(name="Wickets")
        .nlargest(3, "Wickets")
        .sort_values("Wickets", ascending=True)
    )
    top_bowl["display"] = top_bowl.apply(lambda r: f"{r['Bowler']} — {team_abbr(r['Bowling_Team'])}", axis=1)
    top_bowl["color"] = top_bowl["Bowling_Team"].map(lambda t: team_color(t))

    fig_bowl = go.Figure(
        go.Bar(
            x=top_bowl["Wickets"],
            y=top_bowl["display"],
            orientation="h",
            marker_color=top_bowl["color"],
            text=top_bowl["Wickets"],
            textposition="outside"
        )
    )
    fig_bowl.update_layout(
        title="Top 3 Bowlers by Wickets",
        height=260,
        plot_bgcolor=COLORS["bg_dark"],
        paper_bgcolor=COLORS["bg_dark"],
        font_color="white",
        xaxis_title="Wickets",
        yaxis_title=None,
        margin=dict(l=60, r=20, t=40, b=20),
        title_x = 0.5
    )

    plots.append(
        html.Div([
            html.Div(dcc.Graph(figure=fig_bat), style={"width": "48%", "display": "inline-block"}),
            html.Div(dcc.Graph(figure=fig_bowl), style={"width": "48%", "display": "inline-block", "marginLeft": "2%"})
        ])
    )

    # -------------------------------
    # Top 3 Venues (Fixed)
    # -------------------------------
    venue_counts = df["Venue"].value_counts().reset_index()
    venue_counts.columns = ["Venue", "Matches"]
    top3_venues = venue_counts.head(3)
    fig_venues = px.bar(top3_venues, x="Venue", y="Matches", text="Matches", color="Matches", color_continuous_scale="Blues")
    fig_venues.update_traces(textposition="outside")
    fig_venues.update_layout(title="Top 3 Venues (match count)", height=240,
                             plot_bgcolor=COLORS["bg_dark"], paper_bgcolor=COLORS["bg_dark"], font_color="white")
    plots.append(dcc.Graph(figure=fig_venues))

    return kpis, plots

# -------------------------------
# Run App
# -------------------------------
if __name__ == "__main__":
    app.run_server(debug=True, port=8053)
